In [1]:
import pandas as pd
import joblib
import gradio as gr
from surprise import SVD
from surprise import Dataset, Reader

# Load saved artifacts
movies = pd.read_csv("movies.csv")
svd = joblib.load("svd_model.pkl")
cosine_sim = joblib.load("cosine_sim.pkl")

# Hybrid recommendation function
def hybrid_recommendation(user_id, movie_title, top_n=10):
    try:
        idx = movies[movies["Title"] == movie_title].index[0]
    except IndexError:
        return [("Movie not found", 0.0)]

    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:50]

    recommendations = []
    for movie_idx, _ in sim_scores:
        movie_id = movies.iloc[movie_idx]["MovieID"]
        try:
            pred_rating = svd.predict(int(user_id), int(movie_id)).est
            recommendations.append((movies.iloc[movie_idx]["Title"], round(pred_rating, 2)))
        except:
            continue

    recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)
    return recommendations[:top_n]

# Gradio interface
movie_list = movies["Title"].tolist()
interface = gr.Interface(
    fn=hybrid_recommendation,
    inputs=[
        gr.Number(label="User ID", precision=0),
        gr.Dropdown(choices=movie_list, label="Favorite Movie"),
        gr.Slider(minimum=1, maximum=20, step=1, value=10, label="Top N Recommendations")
    ],
    outputs=gr.Dataframe(headers=["Movie", "Predicted Rating"]),
    title="🎬 Movie Recommender System",
    description="Hybrid recommendation using content-based + collaborative filtering (SVD)"
)

if __name__ == "__main__":
    interface.launch()


c:\Users\agash\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
